# Análise do risco de inadimplência dos mutuários

Seu projeto é preparar um relatório para a divisão de empréstimos de um banco. Você precisará descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

Seu relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

[Neste caderno, você recebe dicas e instruções breves e sugestões de raciocínio. Não os ignore, pois eles são projetados para te equipar com a estrutura do projeto e o ajudarão a analisar o que você está fazendo em um nível mais profundo. Antes de enviar seu projeto, certifique-se de remover todas as dicas e descrições fornecidas a você. Em vez disso, faça com que este relatório pareça que você está enviando para seus colegas de equipe para demonstrar suas descobertas - eles não devem saber que você teve qualquer ajuda externa nossa! Para ajudá-lo, colocamos as dicas que você deve remover entre colchetes.]

[Antes de mergulhar na análise de seus dados, explique os propósitos do projeto e as hipóteses que você testará.]

## Abra o arquivo de dados e veja as informações gerais.


In [2]:
import pandas as pd


try:
    credit_score = pd.read_csv('E:/credit_scoring_eng.csv')
except:
    credit_score = pd.read_csv('/datasets/credit_scoring_eng.csv')


## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

Explorando as informações dos dados obtive as seguintes colunas

In [3]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem

credit_score.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# vamos exibir as primeiras 50 linhas

print(credit_score.head(50))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

Primeiramente: o número de dias empregados está com valores negativos em diversos indíviduos. Além disso é um tipo float
o que não faz sentido para uma análise de dias trabalhados.

Segundo: O tipo float para a coluna de renda (total_income) não é propriamente necessária. Equivale ao salário anual
e os centavos não serão relevantes para a análise de crédito nesse caso.

Terceiro: na coluna de propósito do empréstimo (purpose) existem diversos propósitos em duplicata porém com escritas diferentes. Precisa ser feita uma padronização dos motivos e fazer o agrupamento deles.

Quarto: a coluna education precisa de uma padronização de letras minúsculas.

Existem valores ausentes nas colunas days_employed e total_income

Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes

A quantidade de valores ausentes corresponde a um pouco mais de 10% dos dados totais do Dataframe. Se a média dos valores de dias trabalhados com e sem os valores ausentes não for muito discrepante, podemos apenas remover esses valores.


In [5]:
print(credit_score[(credit_score['days_employed'].isna())|credit_score['total_income'].isna()])


       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

Com isso podemos checar que os valores ausentes para days_employed e total_income são os mesmos.

Como estamos criando uma padronização para selecionar se um cliente é ou não é apto a receber o crédito, e esses valores não
irão abaixar muito a média, podemos então remove-los do dataframe sem afetar nossa análise.

In [6]:
# Vamos aplicar várias condições para filtrar dados e observar o número de linhas na tabela filtrada.

print(credit_score[(credit_score['days_employed'].isna()) & credit_score['total_income'].isna()].shape)






(2174, 12)



Os valores ausentes são simétricos. Confirmei que as linhas que possuem valores ausentes são as mesmas tanto para a coluna
de dias empregados quanto para a coluna de renda. 


In [7]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes

filtered_cred_score = credit_score[(credit_score['days_employed'].isna())|credit_score['total_income'].isna()]
print(filtered_cred_score.head(50))



     children  days_employed  dob_years            education  education_id  \
12          0            NaN         65  secondary education             1   
26          0            NaN         41  secondary education             1   
29          0            NaN         63  secondary education             1   
41          0            NaN         50  secondary education             1   
55          0            NaN         54  secondary education             1   
65          0            NaN         21  secondary education             1   
67          0            NaN         52    bachelor's degree             0   
72          1            NaN         32    bachelor's degree             0   
82          2            NaN         50    bachelor's degree             0   
83          0            NaN         52  secondary education             1   
90          2            NaN         35    bachelor's degree             0   
94          1            NaN         34    bachelor's degree    

A porcentagem de valores ausentes é de pouco mais de 10%, o que indica que os valores ausentes não constituem um pedaço
tão grande dos dados, não afetando tanto a distribuição se forem excluídos. Além disso, aparentemente os valores ausentes são
realmente aleatórios, sendo que não encontrei nenhuma correlação entre as colunas que indicasse um padrão.


## Transformação de dados


In [8]:
# Vamos ver todos os valores na coluna de educação para verificar se e quais grafias precisarão ser corrigidas

print(credit_score['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [9]:

credit_score['education'] = credit_score['education'].str.lower()
credit_score = credit_score.replace("graduate degree", "bachelor's degree")

In [10]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos

print(credit_score['education'].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education']


In [11]:
# Vamos ver a distribuição de valores na coluna `children`

print(credit_score['children'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Tem muitas famílias com 20 filhos ou com -1 filhos. Isso implica que houve um erro de digitação, sendo a maior possibilidade que quiseram digitar 2 e 1 e digitaram errado. Por isso troquei os números de filhos pra 2 e 1 e não creio que isso vá 
impactar negativamente na análise.

In [12]:


credit_score['children'] = credit_score['children'].replace(20,2)
credit_score['children'] = credit_score['children'].replace(-1,1)


In [13]:
# Verificando a coluna `children` novamente para ter certeza de que está tudo corrigido

print(credit_score['children'].value_counts())


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [14]:
# Encontrando dados problemáticos em `days_employed`

print(credit_score['days_employed'].unique())


[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


Os valores na coluna 'days_employed' estão negativos, o que não faz sentido. Provavelmente estava configurado pra multiplicar
o valor inserido por -1. Por isso o valor certo tem que ser positivo, então mudei pra valores absolutos.
Além disso, os valores estão em float, e deveriam ser inteiros, pois não existe um valor float de um dia.


In [15]:
credit_score['days_employed'] = abs(credit_score['days_employed'])

In [16]:
# Verificando o resultado

print(credit_score['days_employed'])


0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


In [17]:
# Verificando `dob_years` para valores suspeitos e contando a porcentagem

print(credit_score['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Existe um valor de idade = 0 na coluna. Nesse caso o mais viável é calcular a média e subsitituir os valores de idade 0
pela média.

In [18]:
credit_score['dob_years'] = (credit_score['dob_years'].replace(0,credit_score['dob_years'].mean().astype('int')))

In [19]:
print(credit_score['dob_years'].value_counts())

#A média de idade é 43 anos.


35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [20]:
# Vamos ver os valores da coluna

print(credit_score['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [21]:
#Nesse caso, pra esse estudo, creio que married e civil partnership contem como sendo da mesma categoria, logo podem ser 
#agrupados

credit_score['family_status'] = credit_score['family_status'].replace('civil partnership', 'married')

In [22]:

print(credit_score['family_status'].unique())

['married' 'widow / widower' 'divorced' 'unmarried']


In [23]:
# Vamos ver os valores na coluna

print(credit_score['gender'].unique())

['F' 'M' 'XNA']


In [24]:
#Não existe o genero XNA, mas não acho que seja um problema nesse caso, podemos ignorar esse valor problemático

In [25]:
print(credit_score['gender'].unique())

['F' 'M' 'XNA']


In [26]:
# Vamos ver os valores na coluna

print(credit_score['income_type'].value_counts())


employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


In [27]:

#Para essa coluna os tipos de renda importam. Os únicos que podem ser agrupados são entrepreneur com business e civil servant
#com employee

credit_score['income_type'] = credit_score['income_type'].replace('entrepreneur', 'business')
credit_score['income_type'] = credit_score['income_type'].replace('civil servant', 'employee')

In [28]:
print(credit_score['income_type'].value_counts())


employee                       12578
business                        5087
retiree                         3856
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


In [29]:
credit_score.duplicated().sum()

#Existem 74 linhas duplicadas, e elas podem ser removidas sem problemas pois não irão afetar a análise final


74

In [30]:
credit_score = credit_score.drop_duplicates()

In [31]:
# Última verificação se temos duplicatas

credit_score.duplicated().sum()

0

In [32]:
credit_score.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21451 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21451 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21451 non-null  int64  
 3   education         21451 non-null  object 
 4   education_id      21451 non-null  int64  
 5   family_status     21451 non-null  object 
 6   family_status_id  21451 non-null  int64  
 7   gender            21451 non-null  object 
 8   income_type       21451 non-null  object 
 9   debt              21451 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21451 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


O dataframe inicial possuia 21525 linhas. Após filtragem e seleção dos melhores dados para serem trabalhados terminamos com um dataframe com 21451 linhas.

# Trabalhando com valores ausentes

### Restaurar valores ausentes em `total_income`

As colunas que exigem correção são as colunas 'days_employed' e 'total_income'. Para resolver esse problema vou substituir os valores ausentes pela média ou pela mediana, dependendo da que manter um valor mais próximo da realidade, para cada uma das colunas.



In [33]:
# Vamos escrever uma função que calcule a categoria de idade

def age_group(age):
    if age <= 24:
        return 'young adult'
    if age <= 60:
        return 'adult'
    return 'elder'

In [34]:
# Testando se a função funciona

credit_score['age_group'] = credit_score['dob_years'].apply(age_group)
print(credit_score['age_group'].value_counts())

adult          18450
elder           2126
young adult      875
Name: age_group, dtype: int64


In [35]:
credit_score_xna = credit_score.dropna()
print(credit_score_xna.head(50))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0     926.185831         27    bachelor's degree             0   
6          0    2879.202052         43    bachelor's degree             0   
7          0     152.779569         50  secondary education             1   
8          2    6929.865299         35    bachelor's degree             0   
9          0    2188.756445         41  secondary education             1   
10         2    4171.483647         36    bachelor's degree             0   
11         0     792.701887         40  secondary education             1   

In [36]:
#Os fatores mais impactantes na renda total de alguém são o tipo de educação que a pessoa teve e também o tipo de renda dela.
#Desse modo, vou calcular a média para cada nível de educação e a média para cada tipo de fonte de renda.

print(credit_score_xna.loc[credit_score_xna['education_id']==0]['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['education_id']==1]['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['education_id']==2]['total_income'].mean())

#Os valores foram exatamente os esperados. O grupo com maior renda é o que possui uma graduação em faculdade, seguido 
# pelo grupo que possui algum nível de aprendizado em faculdade e por último os que possuem nível escolar

print(credit_score_xna['income_type'].unique())
print(credit_score_xna.loc[credit_score_xna['income_type']=='employee']['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['income_type']=='retiree']['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['income_type']=='business']['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['income_type']=='unemployed']['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['income_type']=='student']['total_income'].mean())
print(credit_score_xna.loc[credit_score_xna['income_type']=='paternity / maternity leave']['total_income'].mean())

#Aqui novamente foram os valores esperados. A maior renda é de quem possui um negócio, seguido por empregados, aposentados,
#desempregados, estudantes e pais que recebem pensão

#Também é possivel fazer com um for
for i in credit_score_xna['income_type'].unique():
    print(credit_score_xna.loc[credit_score_xna['income_type'] == i]['total_income'].mean())


33142.80243405428
24594.50303709925
29045.443644444447
['employee' 'retiree' 'business' 'unemployed' 'student'
 'paternity / maternity leave']
25997.252501147803
21940.39450304967
32397.165025557017
21014.360500000003
15712.26
8612.661
25997.252501147803
21940.39450304967
32397.165025557017
21014.360500000003
15712.26
8612.661


In [37]:
print(credit_score_xna.loc[credit_score_xna['education_id']==0]['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['education_id']==1]['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['education_id']==2]['total_income'].median())

# Os valores foram exatamente os esperados. O grupo com maior renda é o que possui uma graduação em faculdade, seguido 
# pelo grupo que possui algum nível de aprendizado em faculdade e por último os que possuem nível escolar

print(credit_score_xna['income_type'].unique())
print(credit_score_xna.loc[credit_score_xna['income_type']=='employee']['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['income_type']=='retiree']['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['income_type']=='business']['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['income_type']=='unemployed']['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['income_type']=='student']['total_income'].median())
print(credit_score_xna.loc[credit_score_xna['income_type']=='paternity / maternity leave']['total_income'].median())

print('')

#também é possível usar um for se forem muitos valores únicos
for i in credit_score_xna['income_type'].unique():
    print(credit_score_xna.loc[credit_score_xna['income_type'] == i]['total_income'].median())

28054.531000000003
21836.583
25618.464
['employee' 'retiree' 'business' 'unemployed' 'student'
 'paternity / maternity leave']
22927.388
18962.318
27583.36
21014.360500000003
15712.26
8612.661

22927.388
18962.318
27583.36
21014.360500000003
15712.26
8612.661


Eu fiz as comparações entre o tipo de renda e para o tipo de educação que cada pessoa teve, pra analisar se a renda de cada individuo estava diretamente ligada à sua educação e seu tipo de renda.



As características que eu defini que mais impactam na renda são o grau de escolaridade e o tipo de renda. Os valores da mediana são muito abaixo do valor da média para cada grupo diferente, pois eu realizei agrupamentos pra checar a média e a mediana, e eu só poderia escolher a média se a média fosse próxima em todas categorias. Como não é, a médiana é mais condizente com a realidade.

In [38]:
def fill_nan(df, col, parametro1, parametro2):
    credit_score_xna = credit_score.groupby([parametro1, parametro2])[col].median().reset_index()
    credit_score_xna[col] = credit_score_xna[col].fillna(credit_score[col].median())
    
    for i in range(credit_score_xna.shape[0]):
        grupo1 = credit_score_xna.loc[i, parametro1]
        grupo2 = credit_score_xna.loc[i, parametro2]
        valor = credit_score_xna.loc[i, col]
        
        credit_score.loc[(credit_score[parametro1]==grupo1) & (credit_score[parametro2]==grupo2), col] = credit_score.loc[(credit_score[parametro1]==grupo1) & (credit_score[parametro2]==grupo2), col].fillna(valor)
    
    return credit_score

In [39]:
fill_nan(credit_score,'total_income','education_id','income_type')
credit_score.isna().sum()


children               0
days_employed       2100
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
dtype: int64

###  Restaurar valores em `days_employed`

Do mesmo modo, chequei se os grupos de idade mais avançada não possuem mais dias trabalhados em relação aos adultos.

In [40]:
# Distribuição de `days_employed` medianos com base em seus parâmetros identificados

#Os parametros que influenciam nos dias empregados são: o tipo de emprego/renda e o grupo etário que a pessoa pertence. 

print(credit_score_xna['income_type'].value_counts())
for i in credit_score_xna['income_type'].unique():
    print(credit_score_xna.loc[credit_score_xna['income_type'] == i]['days_employed'].median())
    
#Pessoas que são donas do próprio negócio tem mais dias trabalhados.

print(credit_score_xna['age_group'].value_counts())
for i in credit_score_xna['age_group'].unique():
    print(credit_score_xna.loc[credit_score_xna['age_group'] == i]['days_employed'].median())

#Aqui vejo um problema, a mediana naõ reflete os dias trabalhados do grupo de idosos nem dos jovens adultos

employee                       11326
business                        4578
retiree                         3443
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64
1673.3520511213108
365213.3062657312
1546.3332141566746
366413.65274420456
578.7515535382181
3296.7599620220594
adult          16655
elder           1917
young adult      779
Name: age_group, dtype: int64
2020.3878080900597
744.5421298923785
356191.1376668496


In [41]:
# Distribuição de `days_employed` médios com base em seus parâmetros identificados

print(credit_score_xna['income_type'].value_counts())
for i in credit_score_xna['income_type'].unique():
    print(credit_score_xna.loc[credit_score_xna['income_type'] == i]['days_employed'].mean())

#A média de dias trabalhados pessoas que tem um emprego é a maior novamente, e todos outros valores estão bem próximos    
    
print(credit_score_xna['age_group'].value_counts())
for i in credit_score_xna['age_group'].unique():
    print(credit_score_xna.loc[credit_score_xna['age_group'] == i]['days_employed'].mean())
    
#Apesar do número de dias trabalhados de jovens adultos continuar sendo o maior, os dias empregados de adultos fazem muito mais sentido

employee                       11326
business                        4578
retiree                         3443
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64
2450.8412399581803
365003.4912448612
2111.176937329012
366413.65274420456
578.7515535382181
3296.7599620220594
adult          16655
elder           1917
young adult      779
Name: age_group, dtype: int64
44225.724093366334
1282.6504205713256
290708.5548373909


Vou usar as médias, devido aos dias trabalhados de adultos serem mais condinzentes

In [42]:

def fill_nan2(df, col, parametro1, parametro2):
    credit_score_xna = credit_score.groupby([parametro1, parametro2])[col].mean().reset_index()
    credit_score_xna[col] = credit_score_xna[col].fillna(credit_score[col].mean())
    
    for i in range(credit_score_xna.shape[0]):
        grupo1 = credit_score_xna.loc[i, parametro1]
        grupo2 = credit_score_xna.loc[i, parametro2]
        valor = credit_score_xna.loc[i, col]
        
        credit_score.loc[(credit_score[parametro1]==grupo1) & (credit_score[parametro2]==grupo2), col] = credit_score.loc[(credit_score[parametro1]==grupo1) & (credit_score[parametro2]==grupo2), col].fillna(valor)
    
    return credit_score

In [43]:
# Aplicar função ao income_type

fill_nan2(credit_score,'days_employed','income_type','age_group')
credit_score.isna().sum()



children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

Funcionou, não tenho mais valores ausentes



## Categorização de dados

É importante agora categorizar os dados de propósito do empréstimo, pois existem muitos propositos iguais escritos de maneira diferente, assim como um ranqueamento pro tipo de renda de cada pessoa.


In [44]:
print(credit_score['purpose'].value_counts())

wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
housing                                     646
purchase of the house                       646
purchase of the house for my family         638
construction of own property                634
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         618
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [45]:
print(credit_score['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


In [46]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comunss

def proposito(x):
    if 'wedding ceremony' in x:
        return 1
    if 'having a wedding' in x:
        return 1
    if 'to have a wedding' in x:
        return 1
    if 'real estate transactions' in x:
        return 2
    if 'buy commercial real estate' in x:
        return 2
    if 'housing transactions' in x:
        return 2
    if 'buying property for renting out' in x:
        return 2
    if 'transactions with commercial real estate' in x:
        return 2
    if 'purchase of the house' in x:
        return 2
    if 'housing' in x:
        return 2
    if 'purchase of the house for my family' in x:
        return 2
    if 'construction of own property' in x:
        return 2
    if 'property' in x:
        return 2
    if 'transactions with my real estate' in x:
        return 2
    if 'building a real estate' in x:
        return 2
    if 'buy real estate' in x:
        return 2
    if 'purchase of my own house' in x:
        return 2
    if 'building a property' in x:
        return 2
    if 'housing renovation' in x:
        return 2
    if 'buy residential real estate' in x:
        return 2
    if 'buying my own car' in x:
        return 3
    if 'car' in x:
        return 3
    if 'second-hand car purchase' in x:
        return 3
    if 'cars' in x:
        return 3
    if 'to own a car' in x:
        return 3
    if 'buying a second-hand car' in x:
        return 3
    if 'to buy a car' in x:
        return 3
    if 'car purchase' in x:
        return 3
    if 'purchase of a car' in x:
        return 3
    if 'going to university' in x:
        return 4
    if 'supplementary education' in x:
        return 4
    if 'university education' in x:
        return 4
    if 'education' in x:
        return 4
    if 'to get a supplementary education' in x:
        return 4
    if 'getting an education' in x:
        return 4
    if 'profile education' in x:
        return 4
    if 'getting higher education' in x:
        return 4
    if 'to become educated' in x:
        return 4
    return credit_score
    


In [47]:
# Criando uma coluna com as categorias e contando os valores para elas

credit_score['purpose_id'] = credit_score['purpose'].apply(proposito).astype('int')
print(credit_score['purpose_id'].value_counts())
print(credit_score.head(10))



2    10809
3     4306
4     4012
1     2324
Name: purpose_id, dtype: int64
   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0     926.185831         27    bachelor's degree             0   
6         0    2879.202052         43    bachelor's degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35    bachelor's degree             0   
9         0    2188.756445         41  secondary education             1   

  family_status  family_status_id gender income_type  debt  total_income  \
0       marr

In [48]:
# Examinar todos os dados numéricos em sua coluna selecionada para categorização

def income_total_type(income):
    if income <= 10000:
        return 'low income'
    if income <= 30000:
        return 'medium income'
    return 'high income'


In [49]:
# Obtendo estatísticas resumidas para a coluna

credit_score['income_total_id'] = credit_score['total_income'].apply(income_total_type)
print(credit_score.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0     926.185831         27    bachelor's degree             0   
6         0    2879.202052         43    bachelor's degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35    bachelor's degree             0   
9         0    2188.756445         41  secondary education             1   

  family_status  family_status_id gender income_type  debt  total_income  \
0       married                 0      F    employee     0     40620.102   
1       mar

In [50]:
#Criei grupos baseados na quantidade de renda de cada pessoa e defini grupos de tamanho de renda em baixo, media e alta.

In [51]:
print(credit_score['income_total_id'].head(10))

0      high income
1    medium income
2    medium income
3      high income
4    medium income
5      high income
6      high income
7    medium income
8    medium income
9    medium income
Name: income_total_id, dtype: object


In [52]:
# Contando os valores de cada categoria para ver a distribuição

print(credit_score['income_total_id'].value_counts())

medium income    14419
high income       6106
low income         926
Name: income_total_id, dtype: int64


## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [53]:
# Verificando os dados das crianças e do pagamento em dia

def children_debt(row):
    
    children = row['children']
    debt = row['debt']
    
    if children == 0:
        if debt == 0:
            return 'sem filhos e sem dividas'
    if children == 0:
        if debt == 1:
            return 'sem filhos e com dividas'
    if children >= 1:
        if debt == 0:
            return 'com filhos e sem dividas'
    if children >= 1:
        if debt == 1:
            return 'com filhos e com dividas'

credit_score['filhos_inad'] = credit_score.apply(children_debt, axis=1)
print(credit_score['filhos_inad'].value_counts())

# Calculando a taxa de inadimplência com base no número de filhos

#Para calcular a taxa de inadimplência fazemos a comparação entre as duas razões
print(1063/(13026+1063))
print(678/(6684+678))

#A taxa de inadimplência é maior em famílias com filhos.

sem filhos e sem dividas    13026
com filhos e sem dividas     6684
sem filhos e com dividas     1063
com filhos e com dividas      678
Name: filhos_inad, dtype: int64
0.07544893179075875
0.09209453952730236


**Conclusão**

Após análise, verifiquei que a taxa de inadimplência é de 9.2% em famílias com filhos, em comparação com 7.5% em famílias sem filhos

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [54]:
# Verificando os dados de status da família e do pagamento em dia

def familystat_debt(row):
    
    familystat = row['family_status']
    debt = row['debt']
    
    if 'married' in familystat:
        if debt == 0:
            return 'casado e sem dívidas'
    if 'married' in familystat:
        if debt == 1:
            return 'casado e com dividas'
    if 'widow / widower' in familystat:
        if debt == 0:
            return 'viúvo e sem dividas'
    if 'widow / widower' in familystat:
        if debt == 1:
            return 'viúvo e com dividas'
    if 'divorced' in familystat:
        if debt == 0:
            return 'divorciado e sem dividas'
    if 'divorced' in familystat:
        if debt == 1:
            return 'divorciado e com dividas'
    if 'unmarried' in familystat:
        if debt == 0:
            return 'solteiro e sem dividas'
    if 'unmarried' in familystat:
        if debt == 1:
            return 'solteiro e com dividas'
        
credit_score['familia_inad'] = credit_score.apply(familystat_debt, axis=1)
print(credit_score['familia_inad'].value_counts())

# Calculando a taxa padrão com base no status da família

#Calculando a taxa de inadimplência

print(1593/(17704+1593))
print(85/(85+1110))
print(63/(896+63))

casado e sem dívidas        17704
casado e com dividas         1593
divorciado e sem dividas     1110
viúvo e sem dividas           896
divorciado e com dividas       85
viúvo e com dividas            63
Name: familia_inad, dtype: int64
0.08255169197284552
0.07112970711297072
0.06569343065693431


**Conclusão**

As taxas de inadimplência são: 
8.2% para clientes casados
7.1% para clientes divorciados
6.5% para clientes viúvos

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [55]:
# Verifiquei os dados do nível de renda e do pagamento em dia

print(credit_score['income_total_id'].unique())

def incomerank_debt(row):
    
    incomerank = row['income_total_id']
    debt = row['debt']

    if 'low income' in incomerank:
        if debt == 0:
            return 'baixa renda e sem dívidas'
    if 'low income' in incomerank:
        if debt == 1:
            return 'baixa renda e com dividas'
    if 'medium income' in incomerank:
        if debt == 0:
            return 'media renda e sem dividas'
    if 'medium income' in incomerank:
        if debt == 1:
            return 'media renda e com dividas'
    if 'high income' in incomerank:
        if debt == 0:
            return 'alta renda e sem dividas'
    if 'high income' in incomerank:
        if debt == 1:
            return 'alta renda e com dividas'

credit_score['incomerank_inad'] = credit_score.apply(incomerank_debt, axis=1)
print(credit_score['incomerank_inad'].value_counts())


# Calculei a taxa de inadimplência com base no nível de renda

print(58/(58+868))
print(1239/(13180+1239))
print(444/(444+5662))


['high income' 'medium income' 'low income']
media renda e sem dividas    13180
alta renda e sem dividas      5662
media renda e com dividas     1239
baixa renda e sem dívidas      868
alta renda e com dividas       444
baixa renda e com dividas       58
Name: incomerank_inad, dtype: int64
0.06263498920086392
0.08592828906304183
0.07271536193907632


**Conclusão**

As taxas de inadimplência são as seguintes:
6.2% para clientes com baixa renda
8.5% para clientes com média renda
7.2% para clientes com alta renda

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [56]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os

def purpose_debt(row):
    
    purposeid = row['purpose_id']
    debt = row['debt']

    if purposeid == 1:
        if debt == 0:
            return 'casar e sem dívidas'
    if purposeid == 1:
        if debt == 1:
            return 'casar e com dividas'
    if purposeid == 2:
        if debt == 0:
            return 'comprar uma casa e sem dividas'
    if purposeid == 2:
        if debt == 1:
            return 'comprar uma casa e com dividas'
    if purposeid == 3:
        if debt == 0:
            return 'comprar um carro e sem dividas'
    if purposeid == 3:
        if debt == 1:
            return 'comprar um carro e com dividas'
    if purposeid == 4:
        if debt == 0:
            return 'estudar e sem dividas'
    if purposeid == 4:
        if debt == 1:
            return 'estudar e com dividas'

credit_score['purpose_inad'] = credit_score.apply(purpose_debt, axis=1)
print(credit_score['purpose_inad'].value_counts())

#Taxas de inadimplencia

print(186/(186+2138))
print(782/(782+10027))
print(403/(403+3903))
print(370/(370+3642))

comprar uma casa e sem dividas    10027
comprar um carro e sem dividas     3903
estudar e sem dividas              3642
casar e sem dívidas                2138
comprar uma casa e com dividas      782
comprar um carro e com dividas      403
estudar e com dividas               370
casar e com dividas                 186
Name: purpose_inad, dtype: int64
0.08003442340791739
0.07234711814228884
0.09359033906177427
0.09222333000997009


**Conclusão**

As taxas de inadimplência são:
8% para quem usa o dinheiro pra casar
7.2% pra quem usa o dinheiro pra comprar uma casa
9.3% pra quem usa o dinheiro pra comprar um carro
9.2% pra quem usa o dinheiro pra estudar

# Conclusão Geral 

O primeiro ponto a ser levantado é a grande presença de dados ausentes. Após descobrir como lidar com os dados ausentes, pude realizar comparações que levavam em consideração o tipo de emprego de cada pessoa, a característica familiar (casado, se tem filhos, etc) e relacionar como cada tipo de informação estava diretamente vinculado ao quanto cada individuo recebia. 
Com isso pude definir alguns clientes ideais para receber empréstimo:

Clientes com alta renda, alto grau de escolaridade, sem filhos, casados, e com propósito de comprar uma casa irão possuir uma menor taxa de inadimplência dentre todos analisados.